In [1]:
from transformers.utils import logging

In [7]:
help(GPT2Tokenizer.from_pretrained)

Help on method from_pretrained in module transformers.tokenization_utils_base:

from_pretrained(pretrained_model_name_or_path: Union[str, os.PathLike], *init_inputs, cache_dir: Union[str, os.PathLike, NoneType] = None, force_download: bool = False, local_files_only: bool = False, token: Union[bool, str, NoneType] = None, revision: str = 'main', **kwargs) method of builtins.type instance
    Instantiate a [`~tokenization_utils_base.PreTrainedTokenizerBase`] (or a derived class) from a predefined
    tokenizer.
    
    Args:
        pretrained_model_name_or_path (`str` or `os.PathLike`):
            Can be either:
    
            - A string, the *model id* of a predefined tokenizer hosted inside a model repo on huggingface.co.
              Valid model ids can be located at the root-level, like `bert-base-uncased`, or namespaced under a
              user or organization name, like `dbmdz/bert-base-german-cased`.
            - A path to a *directory* containing vocabulary files require

In [8]:
import urllib3, socket
from urllib3.connection import HTTPConnection

HTTPConnection.default_socket_options = ( 
    HTTPConnection.default_socket_options + [
    (socket.SOL_SOCKET, socket.SO_SNDBUF, 30000000), 
    (socket.SOL_SOCKET, socket.SO_RCVBUF, 30000000)
    ])

In [12]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cpu")

# Загружаем модель ruGPT от сбера
model_name_or_path = "./rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path, resume_download=True, cache_dir="G:\программирование\BMSTU-search-systems")
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

In [20]:
# prompt engineering 
text = "Вопрос: 'Сколько будет 2 + 7?' \n Ответ: " # работает
# text = " Вопрос: 'Сколько будет 3+3?' \n Ответ: 6 . \n Вопрос: 'Сколько будет 1+9?' \n Ответ: 10 . \n Вопрос: 'Сколько будет 4+2?' \n Ответ:" # ValueError
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
out = model.generate(input_ids, do_sample=False) 

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Вопрос: 'Сколько будет 2 + 7?' 
 Ответ: `2+7=14


1. %pip install transformers
2. загружаем Сберовскую модель.
3. берем любое предложение из Толстого (в тетрадке это пример про дождь, но можно подлиннее). 
4. пытаемся генерировать текст.
5. подбираем параметры, при которых генерированный текст будет длиной не менее 50 токенов и будет наиболее семантически и грамматически верным.
6. сдаем мне текст с описанием параметров.

In [21]:
# возьмем какой-нибудь текст
text = 'За окном дождь. Холодный и противный. Хочется'
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)

In [22]:
# если повторить запуск результат не изменится
# но тут уже нет зацикливания потому что модель смотрит дальше чем два токена в прошлое
out = model.generate(input_ids, do_sample=False, max_length=50)


generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


За окном дождь. Холодный и противный. Хочется плакать.

— Я не могу, — говорит она. — Я не могу.

— Почему?

— Потому что я не могу.




In [23]:
out = model.generate(input_ids, do_sample=True,  
                     top_k=0,  # про это параметр ниже
                     max_length=50)


generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


За окном дождь. Холодный и противный. Хочется уйти на улицу, но правда с грязной головой не получится: слишком много грязи. Пускается дождь не один час. У него даже родинка на голове есть. На минуту возникает странное чувство, что


## top_k
Этот параметр ограничивает количество слов, из которых мы семплируем. 10 - означает, что мы выбирает только из 10 самых вероятных слов. В ячейке выше мы поставили top_k = 0, потому что по умолчанию он стоит 50, а нам нужно было попробовать без него.

Чем больше top_k тем более случайный результат мы получим, но слишком низкий top_k может плохо сказаться на разнообразности.

In [24]:
for top_k in [1,3,10, 30, 100]:

    out = model.generate(input_ids, do_sample=True,  
                     top_k=top_k,  # про это параметр ниже
                     max_length=50)


    generated_text = list(map(tokenizer.decode, out))[0]
    print("### text with top_k - ", top_k)
    print(generated_text)
    print()

### text with top_k -  1
За окном дождь. Холодный и противный. Хочется плакать.

— Я не могу, — говорит она. — Я не могу.

— Почему?

— Потому что я не могу.



### text with top_k -  3
За окном дождь. Холодный и противный. Хочется закрыть окна, чтобы не видеть его, не слышать, не ощущать…

– Я не знаю, что делать, – сказал я, не глядя в ее глаза. – Я

### text with top_k -  10
За окном дождь. Холодный и противный. Хочется пить.

– Ты чего не ешь? – спрашивает бабушка, и мне в ответ только слышно, как шуршание ее халата.

– Да нет, ничего

### text with top_k -  30
За окном дождь. Холодный и противный. Хочется согреться. Но где же та, что согреет? И как мне быть с ней?

А я сижу за столом и пишу, не думая о ней.

Это странно.

### text with top_k -  100
За окном дождь. Холодный и противный. Хочется снять с себя всю одежду и зашвырнуть в окурки.
Вдруг с улицы донесся пронзительный женский крик. Окно распахнулось, и на пороге возник муж в полосатой куртке,



# Сэмплирование с Температурой
Еще случайность можно контролировать с помощью параметра, который называется температура. Температура изменяет распределение - при низком значении температуры вероятности переносятся от низких значений к высоким (распределение заостряется), а при высоком - вероятности переносятся от высоких значений к низким (распределение сглаживается).
https://camo.githubusercontent.com/6c20c0e34ce075adbf6bc0fd757b998e38319f819c5bbdca6e6a2b50b7f7b405/68747470733a2f2f6d69726f2e6d656469756d2e636f6d2f76322f726573697a653a6669743a313430302f312a794a55772d7a6b70546671645456654f4f374f4b6e512e706e67![image.png](attachment:image.png)

Нулевая температура означает, что мы на каждом шаге просто выбираем по argmax(), а очень большая температура будет приводить к полному рандому. Под конкретную задачу температуру нужно подбирать отдельно, можно начать с 0 и постепенно увеличивать, смотря на получаемое разнобразие.

(температурой это называется потому что формула взята из физических уравнений, где этот параметр действительно отвечает за температуру)

In [25]:
for temp in [0.001, 0.1, 0.2, 0.5, 0.7, 1., 1.5, 2., 3., 4., 5.]:

    out = model.generate(input_ids, do_sample=True,  
                     top_k=0, 
                     temperature=temp,
                     max_length=50)


    generated_text = list(map(tokenizer.decode, out))[0]
    print("### text with temp - ", temp)
    print(generated_text)
    print()

### text with temp -  0.001
За окном дождь. Холодный и противный. Хочется плакать.

— Я не могу, — говорит она. — Я не могу.

— Почему?

— Потому что я не могу.



### text with temp -  0.1
За окном дождь. Холодный и противный. Хочется закрыть окно и лечь спать. Но я не могу. Я не могу спать. Я не могу спать. Я не могу спать. Я не могу спать. Я не могу спать. Я не могу

### text with temp -  0.2
За окном дождь. Холодный и противный. Хочется спать.

— Пойду, — говорю я. — У меня есть кое-что, что тебе нужно.

— Что? — спрашивает она.


### text with temp -  0.5
За окном дождь. Холодный и противный. Хочется домой.

— К нам гости, — говорит тетя Аня. — Гости, а ты не одета.

— Не беда, — говорю я. —

### text with temp -  0.7
За окном дождь. Холодный и противный. Хочется тепла и уюта. Но нет, этого не будет.

Егор для меня не просто друг, а очень близкий человек. Он помогает мне справляться со всеми трудностями, с которыми мне приходится

### text with temp -  1.0
За окном дождь. Холодный

# Beam Search
У подходов выше есть недостаток - на каждом шаге выбирается только 1 слово и этот выбор нельзя изменить, поэтому 1 неверно выбраное слово можно испортить весь текст и это сложно проконтролировать температурой и топ-к.

С этим может помочь beam-search (поиск пучком). Напомню, что в нем по сути происходит одновременная генерация нескольких текстов параллельно и в конце выбирается текст с наибольшей общей вероятностью. Генерировать все возможные варианты невозможно технически (потому что количество вариаций растет очень быстро), поэтому в beam search варианты отсеиваются на каждом шаге так, чтобы количество текущих вариантов было не больше N. Этот параметр N (размер пучка, beam size) настраивается, но поставить его слишком большим не получится, т.к. опять же будет слишком много комбинаций и это увеличит время генерации.

In [26]:
# beam search уже реализован в hg поэтому нужно только задать параметр num_beams
out = model.generate(input_ids, do_sample=True, num_beams=5, top_k=0, max_length=60)

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text.replace('<s>', ' '))


За окном дождь. Холодный и противный. Хочется плакать.

– Не плачь, – говорит мама. – Все будет хорошо.

– Я не знаю, – говорю я.

– Все будет хорошо, – повторяет мама.




In [35]:
text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=5, top_k=100,
                        max_length=75,
                        repetition_penalty=3.5, # штрафуем повторы
                        no_repeat_ngram_size=2, # напрямую говорит, что нграммы такого размера не должны повторяться совсем
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)
print(len(generated_text))


Дождь идет 
И с неба льется вода.
Я не знаю, что мне делать:
Как я буду жить без тебя?

Припев:<s>
Мои твиты Ср, 13:20: RT @Nemtsov_most: Сегодня День памяти жертв политических репрессий https://t.co/pdFPQ5
206


In [38]:
import gc
gc.collect()

541

In [40]:
text = """Кутузов, сопутствуемый своими адъютантами, поехал шагом за """
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=5, top_k=15,
                        max_length=71,
#                         temperature=0.005,
                        repetition_penalty=3.5, # штрафуем повторы
                        no_repeat_ngram_size=2, # напрямую говорит, что нграммы такого размера не должны повторяться совсем
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)
print(len(generated_text))


Кутузов, сопутствуемый своими адъютантами, поехал шагом за рой саранчи.

— Что это значит? — спросил он у своего адъютанта. Адъютант пожал плечами и ничего не ответил. Он знал Кутузова лучше, чем кто бы то ни было на свете. В его голосе слышалась насмешка
255


In [41]:
text = """Кутузов, сопутствуемый своими адъютантами, поехал шагом """
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=5, top_k=15,
                        max_length=71,
#                         temperature=0.005,
                        repetition_penalty=3.5, # штрафуем повторы
                        no_repeat_ngram_size=2, # напрямую говорит, что нграммы такого размера не должны повторяться совсем
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)
print(len(generated_text))


Кутузов, сопутствуемый своими адъютантами, поехал шагом  в сторону Москвы. 

Вслед за Кутузовым ехал генерал-фельдмаршал М.Б. Барклай де Толли со своим штабом. За ними следовали маршалы Даву, Багратион, Коновницын, Милорадович,
227


In [46]:
text = """Кутузов, сопутствуемый своими адъютантами, поехал шагом """

input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
#                         num_beams=5,
                        top_k=13,
                        max_length=90,
                        temperature=0.13,
                        repetition_penalty=1.3, # штрафуем повторы
                        no_repeat_ngram_size=2, # напрямую говорит, что нграммы такого размера не должны повторяться совсем
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)
print(len(generated_text))


Кутузов, сопутствуемый своими адъютантами, поехал шагом 
в сторону Москвы. 

В Москве он пробыл недолго и вернулся в свой полк с известием о том, что Наполеон уже вступил на русскую землю. В тот же день Кутузова назначили командиром 1-й Западной армии (сформированной из остатков разбитой под Смоленском русской армии). Он немедленно приступил к исполнению своих обязанностей: начал подготовку войск для похода против
417


In [50]:
text = """Кутузов, сопутствуемый своими адъютантами, поехал шагом """

input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=5,
                        top_k=10,
                        max_length=250,
#                         temperature=0.1,
                        repetition_penalty=2.5, # штрафуем повторы
                        no_repeat_ngram_size=2, # напрямую говорит, что нграммы такого размера не должны повторяться совсем
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)
print(len(generated_text))


Кутузов, сопутствуемый своими адъютантами, поехал шагом 
в сторону Смоленска. 

Смоленск был оставлен французами в 1813 году после того, как русские войска под командованием М.И. Кутузова разгромили армию Наполеона и освободили Европу от наполеоновского владычества. В 1812 году по приказу императора Александра I город был переименован в Смоленск-на-Днепре в честь победы русских войск над Наполеоном при Бородино. После изгнания французов из Смоленской губернии (1812 год) на его месте была образована губерния с центром в городе Смоленске. Во время Отечественной войны 1812 года смоленские партизаны принимали активное участие в боевых действиях против французских захватчиков.<s>
Памятник жертвам политических репрессий Оригинал взят у в ПАМЯТНИК ЖЕРТВЫ ПОЛИТИЧЕСКИХ РЕПРЕССИЙ

Мемориальный комплекс "Пермь-36", Пермь, ул. Кунгурская, д. 4/1, стр. 1
http://pamyat-naroda.ru/index.php?option=com_contenttask=viewid
917


In [53]:
text = """Подобно морской рыбе, наглотавшейся пресной воды,
подобно речной рыбёшке, плывущей просто так в океан"""

input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
#                         num_beams=5,
                        top_k=10,
                        max_length=250,
                        temperature=0.2,
                        repetition_penalty=2.5, # штрафуем повторы
                        no_repeat_ngram_size=2, # напрямую говорит, что нграммы такого размера не должны повторяться совсем
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)
print(len(generated_text))


Подобно морской рыбе, наглотавшейся пресной воды,
подобно речной рыбёшке, плывущей просто так в океан.<s>	Ошибка воспроизведения видео: Caption Video Unavailable Click here to view original GIF 
.........................Игорь Стрелков о том как он воюет за Новороссию и против киевской хунты... Оригинал взят у in postВ связи с тем что я не могу комментировать все посты на сайте (а их очень много), то буду писать только по сути - обо всем происходящем сегодня вокруг Новороссии. И начну пожалуй со вчерашнего дня, когда ополчение Донбасса перешло границу Украины под прикрытием танков "Оплот". В этот же день был сбит самолет ВВС РФ который летел над территорией ДНР для того чтобы сбить ополченцев из ПВО ополчения ЛНР которые вели огонь прямой наводкой прямо через линию фронта! Так вот вчера вечером ополченцы перешли границы Донецкой области! А это значит они уже контролируют всю территорию Луганской народной республики!!! Я считаю такое событие нельзя было пропустить мимо ушей ни одному че

In [58]:
text = '''Песня Егора Летова, которую тебе надо продолжить:
"Подобно морской рыбе, наглотавшейся пресной воды,
подобно речной рыбёшке, плывущей просто так в океан'''

input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=5,
                        top_k=7,
                        max_length=250,
#                         temperature=0.005,
#                         repetition_penalty=2.5, # штрафуем повторы
#                         no_repeat_ngram_size=2, # напрямую говорит, что нграммы такого размера не должны повторяться совсем
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)
print(len(generated_text))


Песня Егора Летова, которую тебе надо продолжить:
"Подобно морской рыбе, наглотавшейся пресной воды,
подобно речной рыбёшке, плывущей просто так в океан,
подобно птице, сидящей на ветке,
подобно птице, сидящей на ветке,
подобно птице, сидящей на ветке,
подобно птице, сидящей на ветке,
подобно птице, сидящей на ветке,
подобно птице, сидящей на ветке,
подобно птице, сидящей на ветке,
подобно птице, сидящей на ветке,
подобно птице, сидящей на ветке,
подобно птице, сидящей на ветке,
подобно птице, сидящей на ветке,
подобно птице, сидящей на ветке,
подобно птице, сидящей на ветке,
подобно птице, сидящей на ветке,
подобно птице, сидящей на ветке,
подобно птице, сидящей на ветке,
подобно птице, сидящей на ветке,
подобно пти
726
